In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from absbox.local.util import guess_pool_flow_header

In [203]:
#### from absbox.local.china import SPV
from absbox import API
localAPI = API("http://localhost:8081",lang='english',check=False)


assets = [
            ["FixedAsset"
              ,{"start":"2023-11-01","originBalance":1200,"originTerm":2
               ,"residual":200,"period":"Monthly","amortize":"Straight"
               ,"capacity":("Fixed",100)}
              ,{"balance":489.90,"remainTerm":2}]  
    ] 
    

myAssump = ("Pool"
            ,("Fixed",[["2022-01-01",0.1]]
                     ,[["2022-01-01",400]
                      ,["2024-09-01",300]])
            ,None
            ,None)

p = localAPI.runAsset("2021-04-01"
                      ,assets
                      ,poolAssump=myAssump
                      ,read=True)
p[0]

⠋ Connecting engine server -> http://localhost:8081
✅Connected, local lib:0.20.2, server:0.22.3
⠋


,Balance,Depreciation,CumuDepreciation,Unit,Cash
Date,,,,,
2023-12-01,700,500,500,10,4000
2024-01-01,200,500,1000,10,4000


In [201]:
#### from absbox.local.china import SPV
from absbox import API
localAPI = API("http://localhost:8081",lang='english',check=False)


assets = [
            ["FixedAsset"
              ,{"start":"2023-11-01","originBalance":1200,"originTerm":2
               ,"residual":200,"period":"Monthly","amortize":"DecliningBalance"
               ,"capacity":("Fixed",100)}
              ,{"balance":1200,"remainTerm":2}]  
    ] 
    

myAssump = ("Pool"
            ,("Fixed",[["2022-01-01",0.1]]
                     ,[["2022-01-01",400]
                      ,["2024-09-01",300]])
            ,None
            ,None)

p = localAPI.runAsset("2021-04-01"
                      ,assets
                      ,poolAssump=myAssump
                      ,read=True)
p[0]

⠋ Connecting engine server -> http://localhost:8081
✅Connected, local lib:0.20.2, server:0.22.3
⠋


,Balance,Depreciation,CumuDepreciation,Unit,Cash
Date,,,,,
2023-12-01,700,500,500,10,4000
2024-01-01,200,500,1000,10,4000


In [211]:
from absbox.local.generic import Generic
assets = [["FixedAsset"
            ,{"start":"2023-11-01","originBalance":100_0000,"originTerm":120
              ,"residual":10_0000,"period":"Monthly","amortize":"Straight"
              ,"capacity":("Fixed",24*25*120*30)}
            ,{"balance":100_0000,"remainTerm":120}]] 

exps = (("租金",{"type":{"recurFee":["MonthFirst",18000]}})
        ,("人员工资",{"type":{"recurFee":["MonthFirst",25000]}})
        ,("保险",{"type":{"recurFee":["MonthFirst",4500]}}))

waterfall_1 = {"amortizing":[
                 ["payFee","acc01",['租金','人员工资','保险']]
                ,["accrueAndPayInt","acc01",["A"]]
                ,["payPrin","acc01",["A"]]
             ]
             ,"cleanUp":[
                 ["sellAsset",["Current|Defaulted",1.0,0],"acc01"]
                 ,["payPrin","acc01",["EQ"]]
                 ,["payIntResidual","acc01","EQ"]
             ]}

waterfall_2 = {"amortizing":[
                 ["payFee","acc01",['租金','人员工资','保险']]
                ,["accrueAndPayInt","acc01",["A"]]
                ,["payPrin","acc01",["A"]]
                ,["payPrin","acc01",["EQ"]
                  ,{"limit":{"formula":("floorWithZero"
                               ,("substract"
                                 ,("accountBalance","acc01"),("constant",10000)))
                               }}]
               ]
             ,"cleanUp":[
                 ["sellAsset",["Current|Defaulted",1.0,0],"acc01"]
                 ,["payPrin","acc01",["EQ"]]
                 ,["payIntResidual","acc01","EQ"]
             ]}


evStation = Generic(
    "TEST01"
    ,{"cutoff":"2024-01-01","closing":"2024-01-01","firstPay":"2024-02-01"
     ,"payFreq":"MonthEnd","poolFreq":"MonthEnd","stated":"2050-01-01"}
    ,{'assets':assets}
    ,(("acc01",{"balance":0}),)
    ,(("A",{"balance":100_0000
           ,"rate":0.06
           ,"originBalance":100_0000
           ,"originRate":0.00
           ,"startDate":"2024-01-01"
           ,"rateType":{"Fixed":0.06}
           ,"bondType":{"Sequential":None}})
      ,("EQ",{"balance":100_0000
           ,"rate":0.0
           ,"originBalance":100_0000
           ,"originRate":0.00
           ,"startDate":"2024-01-01"
           ,"rateType":{"Fixed":0.00}
           ,"bondType":{"Equity":None}}),
     )
    ,exps
    ,waterfall_2
    ,[["CollectedCash","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

myAssump = ("Pool"
            ,("Fixed",[["2024-01-01",0.05]
                      ,["2025-01-01",0.08]
                      ,["2026-01-01",0.1]]
                     ,[["2024-01-01",0.3]
                      ,["2025-01-01",0.35]
                      ,["2026-01-01",0.4]])
            ,None
            ,None)

p = localAPI.run(evStation,poolAssump=myAssump
                       ,runAssump=[("call",{"afterDate":"2034-07-01"})
                                  ,("report",{"dates":"MonthEnd"})]
                       ,read=True)

⠋0m


In [212]:
p['result']['report']['balanceSheet']

asset                                               \
                 acc01 Pool Performing Pool Defaulted Pool Recovery   
reportDate                                                            
2024-01-31    32400.00        985000.0         7500.0           0.0   
2024-02-29        0.00        977500.0        15000.0           0.0   
2024-03-31        0.00        970000.0        22500.0           0.0   
2024-04-30        0.00        962500.0        30000.0           0.0   
2024-05-31        0.00        955000.0        37500.0           0.0   
...                ...             ...            ...           ...   
2034-03-31  1315389.48        100000.0       892500.0           0.0   
2034-04-30  1267889.48        100000.0       892500.0           0.0   
2034-05-31  1220389.48        100000.0       892500.0           0.0   
2034-06-30  1172889.48        100000.0       892500.0           0.0   
2034-07-31        0.00        100000.0       892500.0           0.0   

             liability                                                       \
                     A         EQ Accured Int:A Accured Int:EQ Fee Due:人员工资   
reportDate                                                                    
2024-01-31  1000000.00  1000000.0       4931.50            0.0         0.00   
2024-02-29   972695.89  1000000.0       4477.06            0.0      7947.37   
2024-03-31   972695.89  1000000.0       9456.62            0.0     15894.75   
2024-04-30   972695.89  1000000.0      14300.11            0.0     23842.12   
2024-05-31   972695.89  1000000.0      19329.73            0.0     31789.50   
...                ...        ...           ...            ...          ...   
2034-03-31        0.00        0.0          0.00            0.0         0.00   
2034-04-30        0.00        0.0          0.00            0.0         0.00   
2034-05-31        0.00        0.0          0.00            0.0         0.00   
2034-06-30        0.00        0.0          0.00            0.0         0.00   
2034-07-31        0.00        0.0          0.00            0.0         0.00   

                                      equity  
           Fee Due:保险 Fee Due:租金   Net Asset  
reportDate                                    
2024-01-31       0.00       0.00  -980031.50  
2024-02-29    1430.53    5722.11  -999772.96  
2024-03-31    2861.06   11444.22 -1019852.54  
2024-04-30    4291.59   17166.33 -1039796.04  
2024-05-31    5722.12   22888.44 -1059925.68  
...               ...        ...         ...  
2034-03-31       0.00       0.00  2307889.48  
2034-04-30       0.00       0.00  2260389.48  
2034-05-31       0.00       0.00  2212889.48  
2034-06-30       0.00       0.00  2165389.48  
2034-07-31       0.00       0.00   992500.00  

[127 rows x 12 columns]

In [143]:
p['bonds']['EQ']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2024-02-01,1000000.00,0,0.0,0,0.0,<PayPrin:EQ>
2024-02-29,1000000.00,0,0.0,0,0.0,<PayPrin:EQ>
2024-03-31,1000000.00,0,0.0,0,0.0,<PayPrin:EQ>
2024-04-30,1000000.00,0,0.0,0,0.0,<PayPrin:EQ>
2024-05-31,1000000.00,0,0.0,0,0.0,<PayPrin:EQ>
...,...,...,...,...,...,...
2034-03-31,994210.52,40000,0.0,0,40000.0,"[<PayPrin:EQ>, <PayYield:EQ>]"
2034-04-30,994210.52,40000,0.0,0,40000.0,"[<PayPrin:EQ>, <PayYield:EQ>]"
2034-05-31,994210.52,40000,0.0,0,40000.0,"[<PayPrin:EQ>, <PayYield:EQ>]"


In [144]:
p['accounts']['acc01']

,balance,change,memo
date,,,
2024-01-01,32400.00,32400.00,<PoolInflow:CollectedCash>
2024-01-01,32400.00,0.00,<PoolInflow:CollectedCash>
2024-02-01,32400.00,0.00,<SeqPayFee:租金人员工资保险>
2024-02-01,27304.11,-5095.89,<PayInt:A>
2024-02-01,0.00,-27304.11,<PayPrin:A>
...,...,...,...
2034-07-31,0.00,0.00,"[<PayPrin:EQ>, <DS:FloorWithZero (Substract [A..."
2034-07-31,-40000.00,-40000.00,<PayYield:EQ>
2034-07-31,60000.00,100000.00,<Liquidation>


In [108]:
p['bonds']['EQ']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2024-02-01,1000000.00,0.00,0.00,0,0.00,<PayPrin:EQ>
2024-02-29,1000000.00,0.00,0.00,0,0.00,<PayPrin:EQ>
2024-03-31,1000000.00,0.00,0.00,0,0.00,<PayPrin:EQ>
2024-04-30,1000000.00,0.00,0.00,0,0.00,<PayPrin:EQ>
2024-05-31,1000000.00,0.00,0.00,0,0.00,<PayPrin:EQ>
...,...,...,...,...,...,...
2030-06-30,89510.52,0.00,38900.00,0,38900.00,<PayPrin:EQ>
2030-07-31,50610.52,0.00,38900.00,0,38900.00,<PayPrin:EQ>
2030-08-31,11710.52,0.00,38900.00,0,38900.00,<PayPrin:EQ>


In [80]:
p['result']['status']

,Date,From,To
0,2024-01-01,PreClosing,Amortizing
1,2034-07-31,Amortizing,Called


In [147]:
p['result']['bonds']

,Balance Defaults,Interest Defaults,Original Balance,Total
EQ,0,0,1000000,0
A,0,0,1000000,0


In [81]:
from absbox.local.util import irr
irr(p['bonds']['EQ'],init=('2024-01-01',-200_0000))

0.05249321398513658

In [84]:
from absbox.local.util import irr
irr(p['bonds']['EQ'],init=('2024-01-01',-100_0000))

0.07849109483570581

In [148]:
from absbox.local.util import irr
irr(p['bonds']['EQ'],init=('2024-01-01',-100_0000))

0.10411577793847183

In [158]:
scenarioMap = {
    "base":("Pool"
            ,("Fixed",[["2024-01-01",0.05]
                      ,["2025-01-01",0.08]
                      ,["2026-01-01",0.1]]
                     ,[["2024-01-01",0.3]
                      ,["2025-01-01",0.35]
                      ,["2026-01-01",0.4]])
            ,None
            ,None)
    ,"lowUtil" : ("Pool"
            ,("Fixed",[["2024-01-01",0.05]
                      ,["2025-01-01",0.05]
                      ,["2026-01-01",0.8]]
                     ,[["2024-01-01",0.3]
                      ,["2025-01-01",0.35]
                      ,["2026-01-01",0.4]])
            ,None
            ,None)
   ,"lowPrice" : ("Pool"
        ,("Fixed",[["2024-01-01",0.05]
                  ,["2025-01-01",0.05]
                  ,["2026-01-01",0.8]]
                 ,[["2024-01-01",0.25]
                  ,["2025-01-01",0.30]
                  ,["2026-01-01",0.35]])
        ,None
        ,None)
}
p = localAPI.run(evStation,poolAssump=scenarioMap
                       ,runAssump=[("call",{"afterDate":"2034-07-01"})]
                       ,read=True)

⠙0m


In [161]:
from absbox.local.util import irr
{k:irr(v['bonds']['EQ'],init=('2024-01-01',-100_0000)) 
 for k,v in p.items()}

{'base': 0.10411577793847183,
 'lowPrice': 0.5140203223046194,
 'lowUtil': 0.5354475438934259}